In [14]:
from ase.atoms import Atoms
from ase.io import write
from ase.visualize import view
import jax
import jax.numpy as jnp
import sys
sys.path.append('..')
import analysis
from configs import haikumace
import datatypes
from input_pipeline_tf import get_datasets
from models import get_first_node_indices
from train import evaluate_step, get_predictions

In [4]:
config, best_state, metrics_for_best_state = analysis.load_from_workdir('../workdirs/mace0')

/home/songk/atomicarchitects/spherical_harmonic_net/qm9_data_tf/


In [5]:
cutoff = 5.0
key = jax.random.PRNGKey(0)
epsilon = 1e-4

qm9_datasets = get_datasets(key, config)
# atoms = qm9_data[1]

/home/songk/atomicarchitects/spherical_harmonic_net/qm9_data_tf/


In [6]:
example_graph = next(qm9_datasets["test"].as_numpy_iterator())
frag = datatypes.Fragment.from_graphstuple(example_graph)
frag = jax.tree_map(jnp.asarray, frag)

In [7]:
atomic_numbers = jnp.array([1, 6, 7, 8, 9])
numbers_to_symbols = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 9: 'F'}

def get_numbers(species: jnp.ndarray):
    numbers = []
    for i in species:
        numbers.append(atomic_numbers[i])
    return jnp.array(numbers)

In [8]:
preds = get_predictions(best_state, frag)

In [18]:
true_focus = get_first_node_indices(frag)
# this is what the current setup is using as the foci anyway; should be changed later

In [28]:
positions_split = frag.nodes.positions.split(true_focus[1:], axis=0)
species_split = frag.nodes.species.split(true_focus[1:], axis=0)

In [38]:
ndx = 8
frag_atoms = Atoms(positions=positions_split[ndx], numbers=get_numbers(species_split[ndx]))
view(frag_atoms, viewer='ngl')

AttributeError: 'super' object has no attribute '_ipython_display_'

In [ ]:
# save to image files instead:
for i, frag_to_write in enumerate(frags):
    write(f'image{i}.png', Atoms(positions=frag_to_write.nodes.positions, numbers=get_numbers(frag_to_write.nodes.species)))